In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 18:24:22,785 INFO: Initializing external client
2025-03-04 18:24:22,785 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:24:23,940 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214686
Fetching data from 2025-02-03 23:24:22.785203+00:00 to 2025-03-04 22:24:22.785203+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.98s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 00:00:00+00:00,2,0
1,2025-02-04 01:00:00+00:00,2,0
2,2025-02-04 02:00:00+00:00,2,0
3,2025-02-04 03:00:00+00:00,2,0
4,2025-02-04 04:00:00+00:00,2,0
...,...,...,...
177915,2025-03-04 18:00:00+00:00,263,132
177916,2025-03-04 19:00:00+00:00,263,99
177917,2025-03-04 20:00:00+00:00,263,86
177918,2025-03-04 21:00:00+00:00,263,76


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177920 entries, 0 to 190207
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         177920 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  177920 non-null  int32                  
 2   rides               177920 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.1 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177920 entries, 0 to 190207
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         177920 non-null  datetime64[us]
 1   pickup_location_id  177920 non-null  int32         
 2   rides               177920 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.1 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,155,80,14,77,31,207,121,78,42,...,72,88,104,7,12,92,109,88,263,2025-02-21 06:00:00
1,0,0,32,2,7,6,13,13,4,5,...,38,4,3,33,47,41,1,49,261,2025-02-21 16:00:00
2,1,0,0,0,0,0,0,0,3,0,...,0,0,0,0,1,0,0,0,155,2025-02-22 19:00:00
3,0,0,0,0,0,0,0,1,0,0,...,1,0,1,2,1,0,0,0,85,2025-02-23 18:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,59,2025-02-09 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,0,0,0,0,0,0,0,0,0,...,2,0,1,0,3,0,0,2,218,2025-02-09 14:00:00
252,0,0,0,0,0,0,0,1,2,0,...,0,0,0,1,0,1,0,0,258,2025-02-13 03:00:00
253,0,0,0,0,1,0,1,3,0,0,...,0,1,0,0,0,4,0,0,36,2025-02-05 18:00:00
254,0,1,0,0,3,0,0,0,0,0,...,0,1,1,1,1,0,0,0,56,2025-03-01 19:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 18:24:46,171 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:24:46,190 INFO: Initializing external client
2025-03-04 18:24:46,190 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:24:46,956 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214686
Fetching data from 2025-02-03 23:24:46.171490+00:00 to 2025-03-04 22:24:46.171490+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.68s) 


In [12]:
current_date

Timestamp('2025-03-04 23:24:46.171490+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04
1,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04
2,3,6,2,0,1,1,2,1,11,7,...,0,1,1,0,0,1,1,1,4,2025-03-04
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,2025-03-04
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-04
252,0,0,0,0,0,0,1,4,2,0,...,1,6,1,1,0,0,0,1,260,2025-03-04
253,1,3,0,0,2,2,0,4,5,16,...,33,43,45,24,22,21,8,10,261,2025-03-04
254,1,3,1,1,3,19,63,126,177,156,...,87,82,103,65,37,16,14,9,262,2025-03-04


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 18:25:08,148 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:25:08,162 INFO: Initializing external client
2025-03-04 18:25:08,164 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:25:08,903 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214686

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,3.0
3,5,0.0
4,6,0.0
...,...,...
251,259,0.0
252,260,0.0
253,261,3.0
254,262,4.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 138, 142, 230,  48, 114, 186,  68, 249,  79])